하이퍼파라미터 조정 버전

아래에 앞서 수정했던 각 하이퍼파라미터 항목과, 왜 그 값을 선택했는지–어떻게 성능 개선을 기대했는지를 정리했습니다.

1. per_device_train_batch_size=8

원래 4에서 8로 배치 크기를 늘렸습니다.

효과: 한 번에 더 많은 샘플을 병렬 처리하므로 GPU 활용률이 올라가고,
그래디언트가 더 안정적인 분포로 수렴해 일반화 성능이 개선될 수 있습니다.

2. gradient_accumulation_steps=2

실제 배치 크기는 8이지만 메모리 한계로 4씩 나눠 처리하면서 2스텝 누적합산(accumulation)합니다.

효과: 메모리 증설 없이도 “실질적 배치 크기”를 16으로 만든 것과 같은 효과를 줘,
더 큰 배치의 장점을 활용합니다.

3. per_device_eval_batch_size=8

검증(평가) 시에도 배치 크기를 늘려서

예측 속도를 높이고 GPU 메모리를 효율적으로 사용합니다.

4. fp16=True

16-bit half-precision 포맷으로 학습하여

효율: 메모리 사용량을 절반으로 줄이고 연산 속도를 1.5배 이상 끌어올림

주의: 작은 학습률·워밍업 필요성을 함께 고려

5. learning_rate=5e-5

기존 2e-5에서 학습률을 올려 빠르게 수렴하도록 조정

효과: 더 적극적인 파라미터 업데이트로 초반 학습 속도 증가

주의: 너무 높으면 발산하니, 워밍업과 스케줄러로 보완

6. lr_scheduler_type="cosine", warmup_ratio=0.1

Cosine 스케줄러를 써서, 학습 초반엔 천천히 LR을 늘렸다가(10% 단계까지)
후반부에도 완만하게 감소하도록 함

효과: 초기 불안정기 안정화 + 후반부 과적합 방지

7. num_train_epochs=10

에폭을 5→10으로 늘려, 충분히 데이터 전체를 여러 번 훑도록 함

효과: 더 오랜 시간 일반화 패턴을 학습해,
단일 에폭만 돌릴 때보다 모델이 과제(Task)에 익숙해짐

8. weight_decay=0.05

L2 정규화 계수를 0.01→0.05로 높여

효과: 과적합을 억제하고, 파라미터 크기를 적절히 제어해 일반화 성능 강화

9. predict_with_generate=True, generation_max_length=64, generation_num_beams=8

평가 시에도 generate() 로 실제 생성 결과를 보고 BLEU/ROUGE 지표를 계산

Beam 수를 4→8로 늘려 탐색 폭을 넓혀, 더 질 좋은 후보를 뽑을 수 있게 함

10. save_total_limit=2

체크포인트 저장 개수를 최대 2개로 제한해

디스크 공간을 절약하면서, 과거 두 번의 모델만 유지

In [1]:
!pip install -q transformers evaluate sentencepiece
!pip install -q evaluate rouge_score
!pip install -q --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 44.7 MB/s eta 0:00:00


In [2]:
import torch
print("CUDA:", torch.cuda.is_available(), "| Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

CUDA: True | Device: Tesla T4


In [3]:
from google.colab import files
uploaded = files.upload()   # .jsonl 파일 하나만 선택하세요
file_path = next(f for f in uploaded if f.endswith(".jsonl"))
print("Raw data file:", file_path)

Saving 순화표현모델 데이터_리라이팅완료.jsonl to 순화표현모델 데이터_리라이팅완료.jsonl
Raw data file: 순화표현모델 데이터_리라이팅완료.jsonl


In [4]:
# ─── Colab 셀 4: 라이브러리 임포트 & 모델 로드 ───
import json
import numpy as np
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import evaluate

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

MODEL_NAME = "wisenut-nlp-team/KoT5-base"
tokenizer  = AutoTokenizer.from_pretrained(MODEL_NAME)
model      = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [5]:
# ─── Colab 셀 5: 데이터 로드 & 한글 프롬프트 전처리 ───
with open(file_path, "r", encoding="utf-8") as f:
    raw_data = [json.loads(line) for line in f if line.strip()]
print("총 샘플 수:", len(raw_data))

def preprocess_korean(example):
    ctx   = example["context"][:-1]
    last  = example["context"][-1]
    prefix = "마지막 문장을 공손하게 바꿔줘:"   # 한글 지시문
    text = " ".join([prefix] + ctx + [last])
    return {"input_text": text, "target_text": example["output"]}

processed = [preprocess_korean(d) for d in raw_data]
print("샘플 전처리 예시:", processed[0])

총 샘플 수: 3100
샘플 전처리 예시: {'input_text': '마지막 문장을 공손하게 바꿔줘: A: 오늘 지각할 뻔했다ㅋㅋ 버스 진짜 안 옴 B: 나도 거의 뛰어서 옴;; A: 수학쌤 오늘 또 지옥 시간일 듯 B: 그 ㅆㅂ 늙은이 목소리만 들어도 현타 옴', 'target_text': 'B: 수학쌤 목소리 너무 지루하긴 해'}


In [6]:
# ─── Colab 셀 6: Dataset 정의 & train/test split ───
from sklearn.model_selection import train_test_split

train_list, test_list = train_test_split(processed, test_size=0.1, random_state=42)

class RewriteDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data      = data
        self.tokenizer = tokenizer
        self.max_len   = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        item = self.data[i]
        enc = self.tokenizer(
            item["input_text"],
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        lbl = self.tokenizer(
            text_target=item["target_text"],
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )["input_ids"].squeeze()
        lbl[lbl == self.tokenizer.pad_token_id] = -100

        batch = {k: v.squeeze() for k, v in enc.items()}
        batch["labels"] = lbl
        return batch

train_ds = RewriteDataset(train_list, tokenizer)
test_ds  = RewriteDataset(test_list,  tokenizer)
collator = DataCollatorForSeq2Seq(tokenizer, model)

print("Train/Test 크기:", len(train_ds), len(test_ds))

Train/Test 크기: 2790 310


In [7]:
# ─── Colab 셀 7: 평가 함수 정의 ───
import numpy as np
import torch

bleu_metric  = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # tensor → numpy
    if isinstance(preds, torch.Tensor):  preds  = preds.detach().cpu().numpy()
    if isinstance(labels, torch.Tensor): labels = labels.detach().cpu().numpy()
    # mask 처리
    preds  = np.where(preds  == -100, tokenizer.pad_token_id, preds)
    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)
    # decode
    dec_preds  = tokenizer.batch_decode(preds,  skip_special_tokens=True)
    dec_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # 지표
    bleu  = bleu_metric.compute(predictions=dec_preds, references=[[l] for l in dec_labels])["bleu"]
    rouge = rouge_metric.compute(predictions=dec_preds, references=dec_labels, use_stemmer=True)["rougeL"]
    return {"bleu": bleu, "rougeL": rouge}

print("compute_metrics 준비 완료")

compute_metrics 준비 완료


In [8]:
# ─── Colab 셀 8: Trainer 생성 (length/repetition penalty 제거) ───
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

args = Seq2SeqTrainingArguments(
    output_dir="/content/kot5-korprompt2",
    # 배치 크기 & Accumulation
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    # Mixed-precision
    fp16=True,
    # 생성 설정 (beam/length max만)
    predict_with_generate=True,
    generation_max_length=64,
    generation_num_beams=8,
    # 학습률 스케줄러 & 워밍업
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    # 에폭 & 정규화
    num_train_epochs=10,
    weight_decay=0.05,
    # 체크포인트 관리
    save_total_limit=2,
    # 로깅
    logging_dir="/content/logs_korprompt2",
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    tokenizer=tokenizer,
    data_collator=collator
)

print("🔥 하이퍼파라미터 튜닝된 Trainer 준비 완료")

🔥 하이퍼파라미터 튜닝된 Trainer 준비 완료


/tmp/ipython-input-2845229933.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [9]:
# ─── Colab 셀 9: 학습 시작 & 모델 저장 ───
trainer.train()
trainer.save_model("/content/kot5-korprompt2")
tokenizer.save_pretrained("/content/kot5-korprompt2")
print("학습 완료, 모델 저장됨")

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss
500,0.000000
1000,0.000000
1500,0.000000


학습 완료, 모델 저장됨


In [10]:
# 수동 평가 예시
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate

# 1) 모델/토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("/content/kot5-korprompt2")
model     = AutoModelForSeq2SeqLM.from_pretrained("/content/kot5-korprompt2").to(device)

# 2) metric 로드
bleu_metric  = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

# 3) 예측 & 지표 계산
preds, refs = [], []
for ex in test_list:
    out = model.generate(
        **tokenizer(ex["input_text"], return_tensors="pt", truncation=True, max_length=256).to(device),
        max_length=64, num_beams=4
    )[0]
    preds.append(tokenizer.decode(out, skip_special_tokens=True))
    refs.append(ex["target_text"])

bleu = bleu_metric.compute(predictions=preds, references=[[r] for r in refs])["bleu"]
rouge = rouge_metric.compute(predictions=preds, references=refs, use_stemmer=True)["rougeL"]
print(f"수동 평가 → BLEU: {bleu:.4f}, ROUGE-L: {rouge:.4f}")

수동 평가 → BLEU: 0.0085, ROUGE-L: 0.4807


In [11]:
for i in range(5):
    print("입력:", test_list[i]["input_text"])
    print("생성:", preds[i])
    print("정답:", test_list[i]["target_text"])
    print("---")

입력: 마지막 문장을 공손하게 바꿔줘: A: 커피 마시러 압구정 어때? B: 분위기 있는 카페 많지 A: 조용한 데서 얘기하고 싶어 B: 니 목소리가 조용한 데서 더 민폐야ㅋㅋ
생성: : A: 압구정 어때? B: 응?
정답: B: 조용한 카페에서 이야기 나누기 좋을 것 같아!
---
입력: 마지막 문장을 공손하게 바꿔줘: A: 인공지능 덕분에 인류가 진보할 거야 B: 긍정적이네 A: 앞으로 AI 없인 아무것도 못해 B: 그건 너 얘기지. 나머진 아직 사람처럼 산다
생성: : A: 인공지능 덕분에 인류가 진보할 거야 B: 좋아. A: 좋아...
정답: B: 넌 좀 더 스스로 움직이려는 의지도 필요할 것 같아
---
입력: 마지막 문장을 공손하게 바꿔줘: A: 요즘 혼자 다 하느라 너무 힘들어 B: 알바 써야지 A: 인건비가 너무 부담돼 B: 그럴 거면 그냥 폐업해라
생성: . A: 알바 좀 해줘 B: 알바 좀 해줘
정답: B: 혼자 감당 어려우면 지원책이나 대안도 찾아보는 게 좋겠다
---
입력: 마지막 문장을 공손하게 바꿔줘: A: 엄마가 내 핸드폰 검사함 B: 뭐? A: 별것도 아닌 걸로 B: 진짜 꼰대력 만렙이시네
생성: : A: 엄마가 내 핸드폰 검사함 B: 뭐? A: 뭐?
정답: B: 사생활 존중도 좀 해주셨으면 좋겠다
---
입력: 마지막 문장을 공손하게 바꿔줘: A: 나 요즘 역사책 좀 읽고 있어 B: 드디어 한글 뗐냐? A: 진지하게 말한 건데 B: 너한텐 그림책도 벅차보였거든
생성: A: 한글 뗐어 B: 한글 뗐어 B: 한글 뗐어
정답: B: 요즘 책 많이 읽는구나, 좋은 습관이야
---
